In [173]:
using Flux, Statistics
using Flux.Data: DataLoader
using Flux: onehotbatch, onecold, @epochs
using Flux.Losses: mse
using Base: @kwdef
using CUDA
using MLDatasets
using Plots
using Flux: chunk
using Images

In [33]:
function getdata(args, device)
    ENV["DATADEPS_ALWAYS_ACCEPT"] = "true"

    # Loading Dataset	
    xtrain, ytrain = MLDatasets.MNIST.traindata(Float32)
    xtest, ytest = MLDatasets.MNIST.testdata(Float32)

    # Reshape Data in order to flatten each image into a linear array
    xtrain = Flux.flatten(xtrain)
    xtest = Flux.flatten(xtest)

    # One-hot-encode the labels
    ytrain, ytest = onehotbatch(ytrain, 0:9), onehotbatch(ytest, 0:9)

    # Create DataLoaders (mini-batch iterators)
    train_loader = DataLoader((xtrain, ytrain), batchsize=args.batchsize, shuffle=true)
    test_loader = DataLoader((xtest, ytest), batchsize=args.batchsize)

    return train_loader, test_loader
end

getdata (generic function with 1 method)

In [16]:
function build_model(; imgsize=(28,28,1), nclasses=10)
    return Chain( Dense(prod(imgsize), 32, relu),
                  Dense(32, nclasses, relu),
                  Dense(nclasses, 32, relu),
                  Dense(32, prod(imgsize)))
end

build_model (generic function with 1 method)

In [127]:
function SSE(y::AbstractVector, ŷ::AbstractVector)
    res =  y - ŷ
    return res'*res
end

SSE (generic function with 1 method)

In [112]:
function SST(y::AbstractVector)
    z =  y .- mean(y)
    return z'*z
end

SST (generic function with 1 method)

In [154]:
function SSE(Y::AbstractMatrix, Ŷ::AbstractMatrix)
    return sum(abs2, (Y - Ŷ), dims=2)
end

SSE (generic function with 2 methods)

In [160]:
function SST(Y::AbstractMatrix)
    return sum(abs2, (Y .- mean(Y, dims=2)), dims=2)
end

SST (generic function with 2 methods)

In [163]:
function R_square(Y, Ŷ)
    return 1 - sum(SSE(Y, Ŷ))/sum(SST(Y))
end

R_square (generic function with 1 method)

In [165]:
function loss_Rsquare(data_loader, model, device)
    ls = 0.0f0
    num = 0
    R_square_mean = 0
    i = 0

    for (x, y) in data_loader
        x, y = device(x), device(y)
        ŷ = model(x)
        num +=  size(x)[end]
        i +=1
        R_square_mean += R_square(x, ŷ)
        ls += mse(ŷ, x)
    end
    
    
    return ls / num, R_square_mean / i
end

loss_Rsquare (generic function with 1 method)

In [194]:
@kwdef mutable struct Args
    η::Float64 = 3e-4       # learning rate
    batchsize::Int = 256    # batch size
    epochs::Int = 50        # number of epochs
    use_cuda::Bool = true   # use gpu (if cuda available)
end

Args

In [195]:
function train(; kws...)
    args = Args(; kws...) # collect options in a struct for convenience

    if CUDA.functional() && args.use_cuda
        @info "Training on CUDA GPU"
        CUDA.allowscalar(false)
        device = gpu
    else
        @info "Training on CPU"
        device = cpu
    end

    # Create test and train dataloaders
    train_loader, test_loader = getdata(args, device)

    # Construct model
    model = build_model() |> device
    ps = Flux.params(model) # model's trainable parameters
    
    ## Optimizer
    opt = ADAM(args.η)
    
    ## Training
    for epoch in 1:args.epochs
        for (x, y) in train_loader
            x, y = device(x), device(y) # transfer data to device
            gs = gradient(() -> mse(model(x), x), ps) # compute gradient
            Flux.Optimise.update!(opt, ps, gs) # update parameters
        end
        
        # Report on train and test
        train_loss, train_Rsquare = loss_Rsquare(train_loader, model, device)
        test_loss, test_Rsquare = loss_Rsquare(test_loader, model, device)
        println("Epoch=$epoch")
        println("  train_loss = $train_loss, train_Rsquare = $train_Rsquare")
        println("  test_loss = $test_loss, test_Rsquare = $test_Rsquare")
    end
    
    return model

end

train (generic function with 1 method)

In [196]:
model = train()

┌ Info: Training on CUDA GPU
└ @ Main In[195]:5


Epoch=1
  train_loss = 0.0002121212, train_Rsquare = 0.19165361
  test_loss = 0.00021660687, test_Rsquare = 0.17374384
Epoch=2
  train_loss = 0.00018134697, train_Rsquare = 0.30878085
  test_loss = 0.00018462786, test_Rsquare = 0.29525033
Epoch=3
  train_loss = 0.00016226491, train_Rsquare = 0.3815792
  test_loss = 0.00016406535, test_Rsquare = 0.37360066
Epoch=4
  train_loss = 0.0001553067, train_Rsquare = 0.40800172
  test_loss = 0.00015690169, test_Rsquare = 0.40087444
Epoch=5
  train_loss = 0.00015151213, train_Rsquare = 0.4224719
  test_loss = 0.00015330143, test_Rsquare = 0.4145648
Epoch=6
  train_loss = 0.00014872417, train_Rsquare = 0.43306705
  test_loss = 0.00015046583, test_Rsquare = 0.425385
Epoch=7
  train_loss = 0.00014601917, train_Rsquare = 0.4433547
  test_loss = 0.00014765424, test_Rsquare = 0.43610287
Epoch=8
  train_loss = 0.00014007243, train_Rsquare = 0.4661302
  test_loss = 0.0001415969, test_Rsquare = 0.4592514
Epoch=9
  train_loss = 0.00013755429, train_Rsquare

Chain(
  Dense(784, 32, relu),                 # 25_120 parameters
  Dense(32, 10, relu),                  # 330 parameters
  Dense(10, 32, relu),                  # 352 parameters
  Dense(32, 784),                       # 25_872 parameters
)                   # Total: 8 arrays, 51_674 parameters, 880 bytes.

In [192]:
function convert_to_image(x, y_size)
    Gray.(permutedims(vcat(reshape.(chunk(x |> cpu, y_size), 28, :)...), (2, 1)))
end

convert_to_image (generic function with 1 method)

In [197]:
samples = sigmoid.(model(gpu(xtest[:,1:16])))
image = convert_to_image(samples, 16)
save("output/manifold.png", image)

0